# Initialisation

In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2regional as br
import rower as rw
import geopandas as gp
import os
import bz2
import json
from pathlib import Path
bd.projects.set_current("Brightway regional tutorial")

# Regionalisation of the Life Cycle Inventory

There are four different LCI locations for which to create geocollections, topocollections and topographies:
1. Countries of the world (FR, US etc...)
2. Locations of ecoinvent (Europe without Switzerland etc...)
3. ROE (Rest of Europe)
4. ROW (Rest of the world)

In case that you have a specific set of locations for your LCI (ex: location of an agricultural field) you also have to create it.

Running ``create_world_collections()``, ``create_ecoinvent_collections()`` and ``create_restofworlds_collection()`` automatically builds the associated ``geocollections``, ``topocollections`` and ``Topography``. However, for RoW, the objects are not fully populated.

## Regionalisation of the countries of the world

In [7]:
br.create_world_collections()

## Regionalisation of the locations of ecoinvent

In [8]:
br.create_ecoinvent_collections()

## Regionalization of ROW

Currently, the rower package only includes the ROW definition until ecoinvent 3.8. This is why you should define the new RoW of ecoinvent 3.10.1, i.e. the excluded locations for each of the RoW.

Before doing it, you have to change the "hash" of the datapackage EI_GENERIC in order to match the one registered by brightway in your environment.

In [9]:
fp = rw.Rower.EI_GENERIC

In [10]:
metadata_fp = os.path.join(fp, "datapackage.json")

In [11]:
metadata_fp

'C:\\Users\\rogyn\\PycharmProjects\\tutorial_brightway_regional_bw2\\.venv\\Lib\\site-packages\\rower\\data\\Rower\\ecoinvent generic\\datapackage.json'

In [12]:
metadata = json.load(open(metadata_fp))

In [13]:
bd.filesystem.md5(os.path.join(fp, metadata["resources"][0]["path"]))

'fcdf48d44c6a1f3827cebba30765e4e7'

In [14]:
metadata["resources"][0]['hash'] = bd.filesystem.md5(os.path.join(fp, metadata["resources"][0]["path"]))

In [15]:
json.dump(metadata, open(metadata_fp, "w", encoding='utf-8'), indent=2, ensure_ascii=False)

Definition of the new RoW of ecoinvent 3.10.1

In [17]:
rw.update_ecoinvent_definitions_RoW(['ecoinvent-3.10.1-cutoff'])

Processing ecoinvent-3.10.1-cutoff
Adding 92 new RoWs


Creating a geopackage of the RoW of ecoinvent 3.10.1

In [18]:
row_geom_path = Path(os.path.join(rw.DATAPATH_RoW,"ecoinvent-3.10.1-cutoff","RoW_ei_3_10_1.gpkg")) # Choose the path in which you want to save RoW geopackage
dp = rw.RowerDatapackage(os.path.join(rw.DATAPATH_RoW, "ecoinvent-3.10.1-cutoff"))
data = dp.read_data()["Rest-of-World definitions"]

In [19]:
br.cg.add_backward_compatible_definitions()

In [ ]:
br.cg.construct_rest_of_worlds(excluded = data, fp = row_geom_path,use_mp = False)

In [21]:
row_geom = gp.read_file(row_geom_path)

In [22]:
row_geom

,name,id,geometry
0,RoW_100,1,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
1,RoW_106,2,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
2,RoW_107,3,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
3,RoW_11,4,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
4,RoW_110,5,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
...,...,...,...
315,RoW_94,316,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
316,RoW_95,317,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
317,RoW_96,318,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."
318,RoW_97,319,"MULTIPOLYGON (((-176.19884 -44.33506, -176.209..."


Creating RoW topomapping (i.e. the mapping between RoW locations and basic subsections of the worlds)

In [23]:
fp = Path(Path(os.path.join(rw.DATAPATH_RoW,"ecoinvent-3.10.1-cutoff","rows-topomapping.json")))
dp = rw.RowerDatapackage(os.path.join(rw.DATAPATH_RoW, "ecoinvent-3.10.1-cutoff"))
data = dp.read_data()["Rest-of-World definitions"]

In [24]:
br.cg.add_backward_compatible_definitions()

In [25]:
br.cg.construct_rest_of_worlds_mapping(excluded = data, fp=fp)

Compressing the topomapping in the bz2 format

In [26]:
fp_bz2 = Path(Path(os.path.join(rw.DATAPATH_RoW,"ecoinvent-3.10.1-cutoff","rows-topomapping.json.bz2")))

In [27]:
with open(fp, "r", encoding="utf-8") as f_in:
    json_str = f_in.read()
with bz2.open(fp_bz2, "wb") as f_bz2:
    f_bz2.write(json_str.encode("utf-8"))

Creating RoW `geocollection`, `topocollection` and `Topography`

In [28]:
br.create_restofworlds_collections(db_name = "ecoinvent-3.10.1-cutoff", row_gpkg_name = "RoW_ei_3_10_1.gpkg")

Creating `rower` 'RoW' geo/topocollections


## Regionalization of ROE

Define the RoE of ecoinvent 3.10.1, i.e. the excluded european locations for each of the RoE.

In [29]:
fp = rw.Roeer.EI_GENERIC

In [30]:
metadata_fp = os.path.join(fp, "datapackage.json")

In [31]:
metadata_fp

'C:\\Users\\rogyn\\PycharmProjects\\tutorial_brightway_regional_bw2\\.venv\\Lib\\site-packages\\rower\\data\\Roeer\\ecoinvent generic\\datapackage.json'

In [32]:
metadata = json.load(open(metadata_fp))

In [33]:
bd.filesystem.md5(os.path.join(fp, metadata["resources"][0]["path"]))

'73792b9af3fc811b105441cc773526b1'

In [34]:
metadata["resources"][0]['hash'] = bd.filesystem.md5(os.path.join(fp, metadata["resources"][0]["path"]))

In [35]:
json.dump(metadata, open(metadata_fp, "w", encoding='utf-8'), indent=2, ensure_ascii=False)

Definition of the RoE of ecoinvent 3.10.1

In [36]:
rw.update_ecoinvent_definitions_RoE(['ecoinvent-3.10.1-cutoff'])

Processing ecoinvent-3.10.1-cutoff
Adding 15 new RoEs


In [37]:
roe_geom_path = Path(os.path.join(rw.DATAPATH_RoE,"ecoinvent-3.10.1-cutoff","RoE_ei_3_10_1.gpkg")) # Choose the path in which you want to save RoW geopackage
dp = rw.RowerDatapackage(os.path.join(rw.DATAPATH_RoE, "ecoinvent-3.10.1-cutoff"))
data = dp.read_data()["Rest-of-Europe definitions"]

In [20]:
br.cg.add_backward_compatible_definitions()

In [21]:
br.cg.construct_rest_of_europes(excluded = data, fp = roe_geom_path,use_mp = False)

WindowsPath('C:/Users/rogyn/PycharmProjects/tutorial_brightway_regional_bw2/.venv/Lib/site-packages/rower/data/Roeer/ecoinvent-3.10.1-cutoff/RoE_ei_3_10_1.gpkg')

In [39]:
roe_geom = gp.read_file(roe_geom_path)

In [40]:
roe_geom

,name,id,geometry
0,RoE_1,1,"MULTIPOLYGON (((-16.0231 30.03229, -16.02725 3..."
1,RoE_10,2,"MULTIPOLYGON (((-16.02302 30.02924, -16.02798 ..."
2,RoE_11,3,"MULTIPOLYGON (((-17.91405 27.84955, -17.99413 ..."
3,RoE_12,4,"MULTIPOLYGON (((-16.0231 30.03229, -16.02725 3..."
4,RoE_13,5,"MULTIPOLYGON (((-16.0231 30.03229, -16.02725 3..."
5,RoE_14,6,"MULTIPOLYGON (((-17.91405 27.84955, -17.99413 ..."
6,RoE_15,7,"MULTIPOLYGON (((-17.91405 27.84955, -17.99413 ..."
7,RoE_2,8,"MULTIPOLYGON (((-17.91405 27.84955, -17.99413 ..."
8,RoE_3,9,"MULTIPOLYGON (((-16.0231 30.03229, -16.02725 3..."
9,RoE_4,10,"MULTIPOLYGON (((-16.0231 30.03229, -16.02725 3..."


Creating RoE topomapping (i.e. the mapping between RoE locations and basic subsections of the worlds)

In [41]:
fp = Path(Path(os.path.join(rw.DATAPATH_RoE,"ecoinvent-3.10.1-cutoff","roes-topomapping.json")))
dp = rw.RowerDatapackage(os.path.join(rw.DATAPATH_RoE, "ecoinvent-3.10.1-cutoff"))
data = dp.read_data()["Rest-of-Europe definitions"]

In [42]:
br.cg.add_backward_compatible_definitions()

In [43]:
br.cg.construct_rest_of_europes_mapping(excluded = data, fp=fp)

Compressing the topomapping in the bz2 format

In [44]:
fp_bz2 = Path(Path(os.path.join(rw.DATAPATH_RoE,"ecoinvent-3.10.1-cutoff","roes-topomapping.json.bz2")))

In [45]:
with open(fp, "r", encoding="utf-8") as f_in:
    json_str = f_in.read()
with bz2.open(fp_bz2, "wb") as f_bz2:
    f_bz2.write(json_str.encode("utf-8"))

Creating RoW `geocollection`, `topocollection` and `Topography`

In [46]:
br.create_restofeuropes_collections(db_name = "ecoinvent-3.10.1-cutoff", roe_gpkg_name = "RoE_ei_3_10_1.gpkg")

Creating `rower` 'RoE' geo/topocollections


# Modifying the names of the RoW locations and of the RoE locations in the databases

For example, for a spectific activity we need to have RoW_100 instead of RoW and for another specific activity, we need RoE_10 instead of RoE

For the RoWs locations

In [47]:
rower_obj = rw.Rower('ecoinvent-3.10.1-cutoff')

In [48]:
rower_obj.load_existing(os.path.join(rw.DATAPATH_RoW, "ecoinvent-3.10.1-cutoff"))
rower_obj.label_RoWs()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:59


4495

For the RoEs locations

In [49]:
roeer_obj = rw.Roeer('ecoinvent-3.10.1-cutoff')

In [50]:
roeer_obj.load_existing(os.path.join(rw.DATAPATH_RoE, "ecoinvent-3.10.1-cutoff"))
roeer_obj.label_RoEs()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


34

# Labelling the activities of the databases

For each activity of the databases, it is necessary to link them to their respective `geocollection`, i.e. in our case "World", "ecoinvent", "RoW" and "RoE". You have to note that the location and geocollection of the biosphere flows will be the same as their related activities. As such the geocollection of 'biosphere3' is an empty list.

In [51]:
bd.databases['biosphere3']['geocollections'] = []
bd.databases.flush()

In [52]:
br.label_activity_geocollections('ecoinvent-3.10.1-cutoff',RoE=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:04


In [53]:
br.label_activity_geocollections('BW2regional_tutorial',RoE=True)

0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Printing the geocollections of the project

In [54]:
br.geocollections

Geocollections dictionary with 5 object(s):
	Freshwater-acidification-country
	RoE
	RoW
	ecoinvent
	world

Looking at the links between databases and geocollections

In [55]:
for name in bd.databases:
    print(name, bd.databases[name]['geocollections'])

biosphere3 []
ecoinvent-3.10.1-cutoff ['RoE', 'RoW', 'ecoinvent', 'world']
BW2regional_tutorial ['world']


# Calculating the intersections between the LCI maps and the LCIA maps

Define the functional unit : 1 MJ of biodiesel from rapeseed

In [56]:
biodiesel_rapeseed = [activity for activity in bd.Database('BW2regional_tutorial') if "Esterification - Rapeseed - MJ" in activity['name']][0]

In [57]:
UF = {biodiesel_rapeseed:1}

Finding the regionalized method

In [58]:
method = [methods for methods in bd.methods if 'IMPACT World+ Midpoint 2.1 for ecoinvent v3.10, regionalized at the country scale' in methods[0]][0]

For calculating the intersections, my advice is to use `pandarus` as this tool use the `Topographies`. It calculates just once the intersections between spatial unit of two `Topographies` linked to two `Geocollections`.

In [59]:
br.calculate_needed_intersections(UF, method,engine = "pandarus")

Merging topographical faces for geocollection RoW
Creating intersection (RoW, Freshwater-acidification-country)
Merging topographical faces for geocollection world
Creating intersection (world, Freshwater-acidification-country)
Merging topographical faces for geocollection RoE
Creating intersection (RoE, Freshwater-acidification-country)
Merging topographical faces for geocollection ecoinvent
Creating intersection (ecoinvent, Freshwater-acidification-country)
